In [2]:
import torch
import numpy as np
import scipy as sp
from operator import itemgetter

# g(\textbf{X}) = 4 sqrt{2} - X_1 - X_2

Number of imtermediate steps , M = 

# Define the performance function

In [17]:
def performace_fn(x):
    g = 4*np.sqrt(2) - x[0] - x[1]
    return g

Define the hyperparameters

In [18]:
p = 0.1                     # percentile cutoff/ imtermediate failure probability
n = 2                       # number of random variables
N_samples = 1000            # number of samples per subset
N_f = int(N_samples*p)      # number of samples in the failure region
N_c = int(1/p)               # number of samples generated from each Markov Chain

Estimating P(F_1)

In [19]:
# using Monte Carlo to find P(F_1)

# generate samples from the standard normal distribution
np.random.seed(0)
x_samples = np.random.normal(size=(n,N_samples))
g_samples = performace_fn(x_samples).reshape(-1,1)
pf1 = N_f/N_samples

# sorting the samples in ascending order
index, g_sort = zip(*sorted(enumerate(g_samples), key=itemgetter(1)))

# check if failure is reached
    # indicator function for g(X) <= 0
I_samples = np.zeros((N_samples,1))
I_samples[g_samples < 0] = 1
g_th1 = g_sort[N_f - 1]                            # threshold value of g(X)
if g_th1 <= 0:
    print('Failure is reached in the first subset')
    g_th1 = 0
    pf1 = I_samples.sum()/N_samples

g_x1 = g_samples[index[0:N_f], : ]
x1 = x_samples[:, index[0:N_f]]


In [20]:
g_track = g_th1             # keeping track of final sample in the failure region
N_subsets = 1               # number of subsets

## Propagating chains in the subsequent failure regions
Propagating all the Markov Chains sequentially

In [35]:

nDim = 2 # number of dimensions of the problem (number of random variables)
p = 0.1 # intermediate failure probability
N = 1000000 # number of samples per intermediate level
burnin = 0 # burn-in for MCMC (modified Metropolis), 0 in the original subsim

Nt = int(N*p); # number of seeds for the next level

nj = np.zeros((nDim,1))
no_calls = 0 # number of performance function calls

no_rep = 1 # how many independent subsim runs?
for count in range(0,no_rep):
    L = 1 
    nf = 0
    x = np.random.randn(nDim,N)
    y = performace_fn(x) # PERFORMANCE FUNCTION 
    y = y.reshape(-1,1)
# % G(x) should be able to take N number of nDim-dimensional samples in the 
# % form of a nDim x N matrix (the matrix x above) and output N performance 
# % function values. If it cannot, i.e. performance function can only take as
# % input one nDim-dimensional sample and output one number, loop the above st-
# % atement N times for each sample in the matrix x (which is nDim x N) so that
# % y ultimately is a vector of N values.
    # nf = []
    # nf.append(None)
    no_calls = no_calls + N
    pf = []
    while nf/N < p: # loop until the failure region has at least Nt samples, i.e. intermediate threshold value lies in the failure region
        L = L + 1
        ind, y_sort = zip(*sorted(enumerate(y), key=itemgetter(1)))
        # [y_sort,ind] = np.argsort(y,'ascend')
        y_sort = np.array(y_sort)[1:Nt].reshape(-1,1)
        # print(y_sort.shape)
        # print(L)

        yL = y_sort[-1]
        x_sort = x[:,ind[0:Nt]]
        
        mc = np.zeros((nDim, int(1/p) , Nt))
        x = np.zeros((nDim,N))
        y_s = np.zeros(N)
        cont = 1
        for i in range(0,Nt-1): # loop for each seed in the intermediate failure region
            y = np.ones(int(1/p) + burnin)
            mc[:,1,i] = x_sort[:,i]
            for k in range(0,int(1/p) + burnin - 1): # each chain originating from a seed has length (1/p - 1); burnin = 0
                y[0] = y_sort[i]
                nj = mc[:,k,i].reshape(-1,1)
                prop = nj + np.random.randn(nDim,1)
                met_ratio = np.random.normal(prop)/np.random.normal(mc[:,k,i])
                met_satis = 1*(np.random.rand(nDim,1)<met_ratio)
                mc[met_satis, k+1, i] = prop[met_satis].squeeze()
                nj[met_satis] = prop[met_satis]
                
                if (met_satis==np.zeros((nDim,1))).all(): ## ALL OF THE METROPOLIS CONDITIONS?
                    perf_val = y[k]
                else:
                    no_calls = no_calls + 1
                    perf_val = performace_fn(nj) #PERFORMANCE FUNCTION
                
                
                if (perf_val<yL).any(): ## ANY OF THE PERF VALUES?
                    mc[:,k+1,i] = nj.squeeze()
                    y[k+1] = perf_val.reshape(-1)
                else:
                    mc[:,k+1,i] = mc[:,k,i]
                    y[k+1] = y[k]
                
            cont_end = cont + int(1/p) - 1
            x[:,cont:cont_end] = mc[:,burnin+1:,i]
            y_s[cont:cont_end] = y[burnin+1:]
            cont = cont + int(1/p)
        y = y_s
        nf = len(y[y<0])
    prob = p**(L-1) * nf/N
    pf.append(prob)
    # pf[count] = p**(L-1) *nf[L]/N
pf = np.array(pf)
avgg = np.mean(pf)
covv = np.std(pf)/np.mean(pf)


/var/folders/f6/9mr1g0xj6mqf2jl8_cvr2lwc0000gn/T/ipykernel_24610/4173237853.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[0] = y_sort[i]
/var/folders/f6/9mr1g0xj6mqf2jl8_cvr2lwc0000gn/T/ipykernel_24610/4173237853.py:64: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[k+1] = perf_val.reshape(-1)


In [36]:
avgg

0.0023487700000000005

In [34]:
L

3